<a href="https://colab.research.google.com/github/tejasrajanna/PredictThis_Olympics/blob/main/Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [515]:
import numpy as np
import pandas as pd
import plotly.express as px

Converting all the xlsx to csv and storing the files

In [516]:
read_file = pd.read_excel('/content/Athletes.xlsx')
read_file.to_csv (r'/content/Athletes.csv', index = None, header=True)
read_file = pd.read_excel('/content/Coaches.xlsx')
read_file.to_csv (r'/content/Coaches.csv', index = None, header=True)
read_file = pd.read_excel('/content/EntriesGender.xlsx')
read_file.to_csv (r'/content/EntriesGender.csv', index = None, header=True)
read_file = pd.read_excel('/content/Medals.xlsx')
read_file.to_csv (r'/content/Medals.csv', index = None, header=True)
read_file = pd.read_excel('/content/Teams.xlsx')
read_file.to_csv (r'/content/Teams.csv', index = None, header=True)
read_file = pd.read_excel('/content/Population data Assignment 1 Predict This.xlsx')
read_file.to_csv (r'/content/Population data Assignment 1 Predict This.csv', index = None, header=True)
df_athletes=pd.read_csv('/content/Athletes.csv')
df_coaches=pd.read_csv('/content/Coaches.csv')
df_gender=pd.read_csv('/content/EntriesGender.csv')
df_medals=pd.read_csv('/content/Medals.csv')
df_teams=pd.read_csv('/content/Teams.csv')
df_custom=pd.read_csv('/content/Population data Assignment 1 Predict This.csv')

# Data Cleaning

Rough idea of all datasets

In [517]:
df_athletes.head()

,Name,NOC,Discipline
0,AALERUD Katrine,Norway,Cycling Road
1,ABAD Nestor,Spain,Artistic Gymnastics
2,ABAGNALE Giovanni,Italy,Rowing
3,ABALDE Alberto,Spain,Basketball
4,ABALDE Tamara,Spain,Basketball


In [518]:
df_coaches.head()

,Name,NOC,Discipline,Event
0,ABDELMAGID Wael,Egypt,Football,NaN
1,ABE Junya,Japan,Volleyball,NaN
2,ABE Katsuhiko,Japan,Basketball,NaN
3,ADAMA Cherif,Côte d'Ivoire,Football,NaN
4,AGEBA Yuya,Japan,Volleyball,NaN


In [519]:
df_gender.head()

,Discipline,Female,Male,Total
0,3x3 Basketball,32,32,64
1,Archery,64,64,128
2,Artistic Gymnastics,98,98,196
3,Artistic Swimming,105,0,105
4,Athletics,969,1072,2041


In [520]:
df_medals.rename(columns = {'Team/NOC':'Country'}, inplace = True)
df_medals.head()

,Rank,Country,Gold,Silver,Bronze,Total,Rank by Total
0,1,United States of America,39,41,33,113,1
1,2,People's Republic of China,38,32,18,88,2
2,3,Japan,27,14,17,58,5
3,4,Great Britain,22,21,22,65,4
4,5,ROC,20,28,23,71,3


In [521]:
df_teams.head()

,Name,Discipline,NOC,Event
0,Belgium,3x3 Basketball,Belgium,Men
1,China,3x3 Basketball,People's Republic of China,Men
2,China,3x3 Basketball,People's Republic of China,Women
3,France,3x3 Basketball,France,Women
4,Italy,3x3 Basketball,Italy,Women


In [522]:
df_custom.head()

,Country,Population,Independence year,Annual Sports Budget
0,USA,328200000,1776,NaN
1,China,1390000000,1949,NaN
2,Japan,126300000,660,NaN
3,Great Britain,66600000,1776,NaN
4,Russia (ROC),144400000,1990,NaN


Rectifying country names in custom dataset


In [523]:
def func(x):
  if x=='USA':
    return 'United States of America'
  elif x=='China':
    return "People's Republic of China"
  elif x=='Russia (ROC)':
    return 'ROC'
  else:
    return x

df_custom["Country"]=df_custom['Country'].apply(func)
df_custom.head()

,Country,Population,Independence year,Annual Sports Budget
0,United States of America,328200000,1776,NaN
1,People's Republic of China,1390000000,1949,NaN
2,Japan,126300000,660,NaN
3,Great Britain,66600000,1776,NaN
4,ROC,144400000,1990,NaN


Checking for NANs in each dataframe

In [524]:
print(df_athletes.isna().sum().sum())
print(df_coaches.isna().sum().sum())
print(df_gender.isna().sum().sum())
print(df_medals.isna().sum().sum())
print(df_teams.isna().sum().sum())
print(df_custom.isna().sum().sum())

0
145
0
0
0
15


Only the Coaches and Custom (provided by PredictThis) dataset has NANs

In [525]:
print(df_coaches.shape)
df_coaches.info()
df_coaches.drop('Event',axis=1, inplace=True)
#As this dataset is never used for this particular assignment, dropping a column is acceptable

(394, 4)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 394 entries, 0 to 393
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Name        394 non-null    object
 1   NOC         394 non-null    object
 2   Discipline  394 non-null    object
 3   Event       249 non-null    object
dtypes: object(4)
memory usage: 12.4+ KB


In [526]:
print(df_custom.shape)
df_custom.info()
df_custom.drop('Annual Sports Budget',axis=1, inplace=True)
#Entire column is null 

(15, 4)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Country               15 non-null     object 
 1   Population            15 non-null     int64  
 2   Independence year     15 non-null     int64  
 3   Annual Sports Budget  0 non-null      float64
dtypes: float64(1), int64(2), object(1)
memory usage: 608.0+ bytes


Removing countries with zero medals

In [527]:
df_medals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93 entries, 0 to 92
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Rank           93 non-null     int64 
 1   Country        93 non-null     object
 2   Gold           93 non-null     int64 
 3   Silver         93 non-null     int64 
 4   Bronze         93 non-null     int64 
 5   Total          93 non-null     int64 
 6   Rank by Total  93 non-null     int64 
dtypes: int64(6), object(1)
memory usage: 5.2+ KB


In [528]:
df_medals.describe()

,Rank,Gold,Silver,Bronze,Total,Rank by Total
count,93.000000,93.000000,93.000000,93.000000,93.000000,93.000000
mean,46.333333,3.655914,3.634409,4.322581,11.612903,43.494624
std,26.219116,7.022471,6.626339,6.210372,19.091332,24.171769
min,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000
25%,24.000000,0.000000,0.000000,1.000000,2.000000,23.000000
50%,46.000000,1.000000,1.000000,2.000000,4.000000,47.000000
75%,70.000000,3.000000,4.000000,5.000000,11.000000,66.000000
max,86.000000,39.000000,41.000000,33.000000,113.000000,77.000000


Minimum value of total is 1 so no countries with zero medals present in the dataset

# Correlation between population of country and medals

In [529]:
df_medals['population']= df_medals["Country"].map(df_custom.set_index('Country')['Population'])
df_medals.head()

,Rank,Country,Gold,Silver,Bronze,Total,Rank by Total,population
0,1,United States of America,39,41,33,113,1,3.282000e+08
1,2,People's Republic of China,38,32,18,88,2,1.390000e+09
2,3,Japan,27,14,17,58,5,1.263000e+08
3,4,Great Britain,22,21,22,65,4,6.660000e+07
4,5,ROC,20,28,23,71,3,1.444000e+08


In [530]:
df_medals.corr(method='pearson')

,Rank,Gold,Silver,Bronze,Total,Rank by Total,population
Rank,1.000000,-0.655477,-0.597835,-0.637166,-0.655877,0.873539,-0.491373
Gold,-0.655477,1.000000,0.929046,0.862425,0.970840,-0.626012,0.681307
Silver,-0.597835,0.929046,1.000000,0.862646,0.969439,-0.632831,0.593839
Bronze,-0.637166,0.862425,0.862646,1.000000,0.941941,-0.745573,0.234997
Total,-0.655877,0.970840,0.969439,0.941941,1.000000,-0.692450,0.567886
Rank by Total,0.873539,-0.626012,-0.632831,-0.745573,-0.692450,1.000000,-0.479156
population,-0.491373,0.681307,0.593839,0.234997,0.567886,-0.479156,1.000000


Plot to show top 15 countries with reference to Gold Medals

In [531]:
# Creating the Figure instance
fig = px.scatter(df_medals[:15],x="population", y='Gold', color='Country')   
# showing the plot
fig.show()

We can see that number of Gold medals won by each country is strongly related to their population with a few outliers, most noticeable ones being Brazil and Australia.
They share a positive correlation.

Plot to show top 15 countries with reference to Silver Medals

In [532]:
fig = px.scatter(df_medals[:15],x="population", y='Silver', color='Country') 
fig.show()

Number of Silver medals won by each country is closely related to their population with the most obvious outlier once again being Brazil along with Great Britain.

They share a positive correlation but it is slightly weaker than the correlation between Gold Medals and population of each country

Plot to show top 15 countries with reference to Bronze Medals

In [533]:
fig = px.scatter(df_medals[:15],x="population", y='Bronze', color='Country') 
fig.show()

Number of Bronze Medals won by a country is weakly related to its population.
They share a positive correlation and we can clearly see many outliers such as China, Brazil, France, Australia and so on.

Plot to show top 15 countries with reference to Total Medals

In [534]:
fig = px.scatter(df_medals[:15],x="population", y='Total', color='Country') 
fig.show()

Number of Total Medals won by a country is closely related to its population but there are some outliers with Brazil being the biggest culprit yet again along with Australia. 
They share a positive correlation

We can infer that the densely populated countries' total medal tally generally boasts of a large proportion being Gold medals whereas the thinly populated countries medal tally mainly consists of Bronze medals.


# Correlation between number of athletes from each country and medals

In [535]:
print(df_athletes['NOC'].nunique())
pop=df_athletes['NOC'].value_counts()
pop_countries=pop.index.to_series()
print(pop)

206
United States of America         615
Japan                            586
Australia                        470
People's Republic of China       401
Germany                          400
                                ... 
Marshall Islands                   2
Nauru                              2
Mauritania                         2
Saint Kitts and Nevis              2
St Vincent and the Grenadines      2
Name: NOC, Length: 206, dtype: int64


In [536]:
data = {"Countries": pop_countries,
        "Athletes_no": pop}
df_pop = pd.concat(data,
               axis = 1)
df_pop.reset_index(drop=True, inplace= True)
print(df_pop)

                         Countries  Athletes_no
0         United States of America          615
1                            Japan          586
2                        Australia          470
3       People's Republic of China          401
4                          Germany          400
..                             ...          ...
201               Marshall Islands            2
202                          Nauru            2
203                     Mauritania            2
204          Saint Kitts and Nevis            2
205  St Vincent and the Grenadines            2

[206 rows x 2 columns]


In [537]:
df_medals["Athletes_no"] = df_medals["Country"].map(lambda x: data['Athletes_no'].get(x,x))
print(df_medals.head())

   Rank                     Country  ...    population  Athletes_no
0     1    United States of America  ...  3.282000e+08          615
1     2  People's Republic of China  ...  1.390000e+09          401
2     3                       Japan  ...  1.263000e+08          586
3     4               Great Britain  ...  6.660000e+07          366
4     5                         ROC  ...  1.444000e+08          318

[5 rows x 9 columns]


In [538]:
df_medals.corr(method='pearson')

,Rank,Gold,Silver,Bronze,Total,Rank by Total,population,Athletes_no
Rank,1.000000,-0.655477,-0.597835,-0.637166,-0.655877,0.873539,-0.491373,-0.697995
Gold,-0.655477,1.000000,0.929046,0.862425,0.970840,-0.626012,0.681307,0.839912
Silver,-0.597835,0.929046,1.000000,0.862646,0.969439,-0.632831,0.593839,0.796216
Bronze,-0.637166,0.862425,0.862646,1.000000,0.941941,-0.745573,0.234997,0.890906
Total,-0.655877,0.970840,0.969439,0.941941,1.000000,-0.692450,0.567886,0.875115
Rank by Total,0.873539,-0.626012,-0.632831,-0.745573,-0.692450,1.000000,-0.479156,-0.744453
population,-0.491373,0.681307,0.593839,0.234997,0.567886,-0.479156,1.000000,0.246756
Athletes_no,-0.697995,0.839912,0.796216,0.890906,0.875115,-0.744453,0.246756,1.000000


In [539]:
fig=px.scatter(df_medals[:15], x='population', y='Athletes_no', color='Country')
fig.show()

There is no strong relationship between Number of Athletes and Population of Country which shows that quality of athletes is prioritised over quantity.

In [540]:
fig=px.bar(df_medals[:15], x='Rank by Total', y='Athletes_no', color='Country', text='Rank by Total')
fig.show()

We can see that Number of Athletes and Rank by total are closely related with the most glaring outliers being Japan and ROC.
They have a negative correlation which implies that larger the number of athletes, the higher the rank obtained with the caveat being the quality of athletes

# Correlation between Independence Year and Medals

In [541]:
df_medals['independence year']= df_medals["Country"].map(df_custom.set_index('Country')['Independence year'])
df_medals.head()

,Rank,Country,Gold,Silver,Bronze,Total,Rank by Total,population,Athletes_no,independence year
0,1,United States of America,39,41,33,113,1,3.282000e+08,615,1776.0
1,2,People's Republic of China,38,32,18,88,2,1.390000e+09,401,1949.0
2,3,Japan,27,14,17,58,5,1.263000e+08,586,660.0
3,4,Great Britain,22,21,22,65,4,6.660000e+07,366,1776.0
4,5,ROC,20,28,23,71,3,1.444000e+08,318,1990.0


In [542]:
df_medals.corr(method='pearson')

,Rank,Gold,Silver,Bronze,Total,Rank by Total,population,Athletes_no,independence year
Rank,1.000000,-0.655477,-0.597835,-0.637166,-0.655877,0.873539,-0.491373,-0.697995,0.303659
Gold,-0.655477,1.000000,0.929046,0.862425,0.970840,-0.626012,0.681307,0.839912,-0.230540
Silver,-0.597835,0.929046,1.000000,0.862646,0.969439,-0.632831,0.593839,0.796216,0.031113
Bronze,-0.637166,0.862425,0.862646,1.000000,0.941941,-0.745573,0.234997,0.890906,-0.042853
Total,-0.655877,0.970840,0.969439,0.941941,1.000000,-0.692450,0.567886,0.875115,-0.090683
Rank by Total,0.873539,-0.626012,-0.632831,-0.745573,-0.692450,1.000000,-0.479156,-0.744453,0.150046
population,-0.491373,0.681307,0.593839,0.234997,0.567886,-0.479156,1.000000,0.246756,0.121747
Athletes_no,-0.697995,0.839912,0.796216,0.890906,0.875115,-0.744453,0.246756,1.000000,-0.449754
independence year,0.303659,-0.230540,0.031113,-0.042853,-0.090683,0.150046,0.121747,-0.449754,1.000000


Plot to show top 15 countries with reference to Gold Medals

In [543]:
fig = px.scatter(df_medals[:15],x="independence year", y='Gold', color='Country') 
fig.show()

There is a very weak negative correlation between Independence Year of the country and Number of Gold Medals won by it which implies that the earlier a country gained independence, the greater the chance it had to win more number of Gold Medals.
Netherlands, ROC and several countries defy this trend

Plot to show top 15 countries with reference to Silver Medals

In [544]:
fig = px.scatter(df_medals[:15],x="independence year", y='Silver', color='Country') 
fig.show()

Plot to show top 15 countries with reference to Bronze Medals

In [545]:
fig = px.scatter(df_medals[:15],x="independence year", y='Bronze', color='Country') 
fig.show()

Plot to show top 15 countries with reference to Total Medals

In [546]:
fig = px.scatter(df_medals[:15],x="independence year", y='Total', color='Country') 
fig.show()

There is no clear discernible pattern for the relationship between Independence Year of country and Silver, Bronze and Total Medals won by country.
Hence there is little to no correlation between them

# Top 3 Highest Divide

In [547]:
df_gender['Zero']= df_gender.all(axis='columns')
df_gender.head()

,Discipline,Female,Male,Total,Zero
0,3x3 Basketball,32,32,64,True
1,Archery,64,64,128,True
2,Artistic Gymnastics,98,98,196,True
3,Artistic Swimming,105,0,105,False
4,Athletics,969,1072,2041,True


In [548]:
index_names= df_gender.index[ df_gender['Zero']== False ]
df_filt=df_gender.drop(index_names)
df_filt['Divide']= (df_filt['Male'] - df_filt['Female']).abs()
df_filt.head()

,Discipline,Female,Male,Total,Zero,Divide
0,3x3 Basketball,32,32,64,True,0
1,Archery,64,64,128,True,0
2,Artistic Gymnastics,98,98,196,True,0
4,Athletics,969,1072,2041,True,103
5,Badminton,86,87,173,True,1


In [551]:
df_top= df_filt.sort_values(by='Divide', ascending= False)
df_top.head()

,Discipline,Female,Male,Total,Zero,Divide
4,Athletics,969,1072,2041,True,103
45,Wrestling,96,193,289,True,97
9,Boxing,102,187,289,True,85
20,Football,264,344,608,True,80
15,Cycling Road,70,131,201,True,61


The top 3 disciplines which have highest divide in men's entries vs women's entries are Athletics, Wrestling and Boxing.

Elite female athletes are not equal to their male counterpart in terms of physical ability. 
Women’s sports that are identical to men’s sports will never be popular, because men are faster, stronger and more athletic. On the other hand, sports that highlight the different strengths of female athletes are popular. Less popularity leads to gender pay gap which is a huge factor in the divide